In [219]:
# -*- coding: utf-8 -*-

import os
import time
import random

from PIL import Image

from selenium import webdriver

from lib.ocr.utils import get_digest
from lib.ocr.cracker import init_model, crack_thsr
from lib.common.utils import get_chrome_driver, check_folder

from selenium.common.exceptions import NoSuchElementException        
from selenium.webdriver.common.keys import Keys

url = "https://irs.thsrc.com.tw/IMINT/"

opener = get_chrome_driver()
opener.get(url)

export webdriver.chrome.driver=lib/common/../../etc/driver/chromedriver_mac64


In [221]:
# start_station
opener.find_element_by_name("selectStartStation").send_keys(unicode("台北", "UTF8"))

# end_station
opener.find_element_by_name("selectDestinationStation").send_keys(unicode("台中", "UTF8"))

# preferred seat
opener.find_element_by_id("seatRadio1").click()

# preferred booking
opener.find_element_by_id("bookingMethod1").click()

# preferred date
input_field = opener.find_element_by_id('toTimeInputField')

opener.execute_script("arguments[0].value = ''", input_field)
input_field.send_keys('2017/03/01')

# preferred time
opener.find_element_by_name("toTimeTable").send_keys("130P")

# preferred adult amount
opener.find_element_by_name("ticketPanel:rows:0:ticketAmount").send_keys("2")

# preferred children amount
opener.find_element_by_name("ticketPanel:rows:1:ticketAmount").send_keys("2")

In [222]:
# crack the captcha image
retry = 5
while retry > 0:
    try:
        img = opener.find_element_by_id("BookingS1Form_homeCaptcha_passCode")
        location, size = img.location, img.size

        if isinstance(opener, webdriver.Chrome):
            location["x"] *= 2
            location["y"] *= 2
            size["width"] *= 2
            size["height"] *= 2
        else:
            location["x"] += 5

        filepath_screenshot = "/tmp/1.jpg"
        opener.save_screenshot(filepath_screenshot)

        im = Image.open(filepath_screenshot)
        im = im.crop((location["x"], location["y"], location["x"]+size["width"], location["y"] + size["height"]))

        if isinstance(opener, webdriver.Chrome):
            im = im.resize((im.size[0]/2, im.size[1]/2), Image.ANTIALIAS)

        im.show()

        im_filepath = os.path.join("tmp", "{}.jpg".format(get_digest(im)))
        check_folder(im_filepath)
        im.save(im_filepath)

        init_model("thsr")
        answer = crack_thsr(im_filepath, cropped=2)

        submit = opener.find_element_by_name("homeCaptcha:securityCode")
        submit.clear()
        submit.send_keys(answer)

        opener.find_element_by_id("SubmitButton").click()
        time.sleep(random.randint(1, 3))

        if opener.current_url == "https://irs.thsrc.com.tw/IMINT/?wicket:interface=:1::":
            retry = -1
            break
    except NoSuchElementException as nee:
        pass
    
    retry -= 1
    time.sleep(random.randint(1, 5))

In [223]:
trains = []
for field in opener.find_elements_by_xpath("//tr"):
    if field.get_attribute("onMouseOver"):
        trains.append(field.text.split())
        
    for img_field in field.find_elements_by_tag_name("img"):
        trains[-1].append(img_field.get_attribute("src"))
        
    
    for radio_field in field.find_elements_by_name("TrainQueryDataViewPanel:TrainGroup"):
        trains[-1].append(radio_field)
        
print trains

[[u'639', u'12:46', u'13:46', u'1:00', u'https://irs.thsrc.com.tw/IMINT/resources/tw.com.mitac.webapp.thsr.viewer.panel.LightLoadedTrainQueryDataViewPanel/irs_ind_eb2_zh_TW.gif', <selenium.webdriver.remote.webelement.WebElement (session="a4f4e891d2d97216344d760a12111b3f", element="0.7386164939581004-18")>], [u'829', u'13:11', u'14:15', u'1:04', u'https://irs.thsrc.com.tw/IMINT/resources/tw.com.mitac.webapp.thsr.viewer.panel.LightLoadedTrainQueryDataViewPanel/irs_ind_ebH_zh_TW.gif', <selenium.webdriver.remote.webelement.WebElement (session="a4f4e891d2d97216344d760a12111b3f", element="0.7386164939581004-20")>], [u'133', u'13:31', u'14:18', u'0:47', u'https://irs.thsrc.com.tw/IMINT/resources/tw.com.mitac.webapp.thsr.viewer.panel.LightLoadedTrainQueryDataViewPanel/irs_ind_eb2_zh_TW.gif', <selenium.webdriver.remote.webelement.WebElement (session="a4f4e891d2d97216344d760a12111b3f", element="0.7386164939581004-22")>], [u'645', u'13:46', u'14:46', u'1:00', u'https://irs.thsrc.com.tw/IMINT/reso

In [224]:
trains[5][-1].click()

opener.find_element_by_name("SubmitButton").click()

In [216]:
if opener.current_url == "https://irs.thsrc.com.tw/IMINT/?wicket:interface=:2::":
    ticket_info = opener.find_element_by_xpath("//table[@class='table_simple']")
    print ticket_info.text
    
    opener.find_element_by_id("idNumber").send_keys("L122760167")
    
    opener.find_element_by_id("mobileInputRadio").click()
    opener.find_element_by_id("mobilePhone").send_keys("0921747196")
    
    opener.find_element_by_name("agree").click()
    opener.find_element_by_id("isSubmit").click()

行程 日期 車次 起程站 到達站 出發時間 到達時間 全票 敬老、孩童、愛心 小計
去程 03/01 837 台北 台中 15:11 16:15 560*2 350*2 TWD 1,820
車廂：標準車廂 票數：全票 2 張 | 孩童票 2 張  總票價 TWD 1,820
